In [20]:
# Import library
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [21]:
# Declare engine variables
user = "root"
password = ""
port = 3306
database = "airline_quality"

In [22]:
# Create engine to query data from database
engine = create_engine(
    "mysql+mysqldb://%s:%s@localhost:%i/%s" % (user, password, port, database)
)

In [23]:
# Retrieve airline passenger satisfaction from database
select_command = "SELECT * FROM airline_passenger_satisfaction"
passenger_satis = pd.read_sql_query(select_command, engine)
passenger_satis.head()

,ID,Gender,Age,Customer Type,Type of Travel,Class,Flight Distance,Departure Delay,Arrival Delay,Departure and Arrival Time Convenience,...,On-board Service,Seat Comfort,Leg Room Service,Cleanliness,Food and Drink,In-flight Service,In-flight Wifi Service,In-flight Entertainment,Baggage Handling,Satisfaction
0,1,Male,48,First-time,Business,Business,821,2,5,3,...,3,5,2,5,5,5,3,5,5,Neutral or Dissatisfied
1,2,Female,35,Returning,Business,Business,821,26,39,2,...,5,4,5,5,3,5,2,5,5,Satisfied
2,3,Male,41,Returning,Business,Business,853,0,0,4,...,3,5,3,5,5,3,4,3,3,Satisfied
3,4,Male,50,Returning,Business,Business,1905,0,0,2,...,5,5,5,4,4,5,2,5,5,Satisfied
4,5,Female,49,Returning,Business,Business,3470,0,1,3,...,3,4,4,5,4,3,3,3,3,Satisfied


In [24]:
# Retrieve airline comments from database
select_command = "SELECT * FROM airline_comments"
passenger_comment = pd.read_sql_query(select_command, engine)
passenger_comment.tail()

,ID,Airline Name,Verified,Date Flown,Value For Money,Recommend
4100,4332,American Eagle,FALSE,October 2015,1,no
4101,4333,American Eagle,FALSE,September 2015,5,yes
4102,4334,American Eagle,FALSE,October 2015,1,no
4103,4335,American Eagle,FALSE,August 2015,4,yes
4104,4336,American Eagle,FALSE,August 2015,4,yes


In [25]:
# Select values what have airline comments
selected_passenger_satis = passenger_satis[passenger_satis["ID"] <= 4336]

In [26]:
# Perform join 2 datasets
merge_df = pd.merge(
    selected_passenger_satis, passenger_comment, left_on="ID", right_on="ID", how="left"
).reset_index(drop=True)
merge_df

,ID,Gender,Age,Customer Type,Type of Travel,Class,Flight Distance,Departure Delay,Arrival Delay,Departure and Arrival Time Convenience,...,In-flight Service,In-flight Wifi Service,In-flight Entertainment,Baggage Handling,Satisfaction,Airline Name,Verified,Date Flown,Value For Money,Recommend
0,1,Male,48,First-time,Business,Business,821,2,5,3,...,5,3,5,5,Neutral or Dissatisfied,AB Aviation,TRUE,June 2019,2.0,no
1,2,Female,35,Returning,Business,Business,821,26,39,2,...,5,2,5,5,Satisfied,AB Aviation,TRUE,June 2019,2.0,no
2,3,Male,41,Returning,Business,Business,853,0,0,4,...,3,4,3,3,Satisfied,Adria Airways,FALSE,September 2019,1.0,no
3,4,Male,50,Returning,Business,Business,1905,0,0,2,...,5,2,5,5,Satisfied,Adria Airways,TRUE,September 2019,1.0,no
4,5,Female,49,Returning,Business,Business,3470,0,1,3,...,3,3,3,3,Satisfied,Adria Airways,TRUE,September 2019,1.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,4332,Female,40,Returning,Business,Business,3851,212,168,3,...,3,2,2,3,Neutral or Dissatisfied,American Eagle,FALSE,October 2015,1.0,no
4316,4333,Female,18,Returning,Business,Business,3648,0,0,5,...,4,5,3,4,Satisfied,American Eagle,FALSE,September 2015,5.0,yes
4317,4334,Male,41,Returning,Business,Business,618,41,36,4,...,2,4,2,2,Satisfied,American Eagle,FALSE,October 2015,1.0,no
4318,4335,Male,53,Returning,Business,Business,473,0,7,3,...,4,3,4,4,Satisfied,American Eagle,FALSE,August 2015,4.0,yes


In [27]:
# Convert customer Type value to dimension table
merge_df["customer_type"] = np.where(merge_df["Customer Type"] == "First-time", 1, 2)

In [28]:
# Convert Type of Travel to dimension table
merge_df["travel_type"] = np.where(merge_df["Type of Travel"] == "Business", 1, 2)

In [29]:
# Convert Class to dimension table
merge_df["class_type"] = np.where(
    merge_df["Class"] == "Business",
    1,
    np.where(merge_df["Class"] == "Economy", 2, 3),
)

In [30]:
# Convert customer satisfaction to dimension table
merge_df["satis_level"] = np.where(
    merge_df["Satisfaction"] == "Neutral or Dissatisfied", 1, 2
)

In [31]:
# Check null value for "Value for Money" variable
merge_df["Value For Money"].isnull().sum()

231

In [32]:
# Fill N/A value for "Value for Money" variable
merge_df["Value For Money"] = merge_df["Value For Money"].fillna(0)

In [33]:
# Check null value for Value For Money variable after filled N/A values
merge_df["Value For Money"].isnull().sum()

0

In [34]:
# Change float datatype into integer data type
merge_df["Value For Money"] = merge_df["Value For Money"].astype(int)

In [49]:
# Grouped by "Date Flown" variable and selected only variable what impact to airline
grouped_df = (
    merge_df.groupby(
        [
            "Date Flown",
            "Gender",
            "Age",
            "customer_type",
            "travel_type",
            "class_type",
            "satis_level",
            "Verified",
            "Recommend",
        ]
    )
    .count()
    .reset_index()
)[
    [
        "Date Flown",
        "Gender",
        "Age",
        "customer_type",
        "travel_type",
        "class_type",
        "satis_level",
        "Verified",
        "Recommend",
    ]
]
grouped_df

,Date Flown,Gender,Age,customer_type,travel_type,class_type,satis_level,Verified,Recommend
0,,Female,7,2,2,1,1,FALSE,yes
1,,Female,7,2,2,2,1,FALSE,yes
2,,Female,8,1,1,3,1,FALSE,yes
3,,Female,8,2,2,2,1,FALSE,yes
4,,Female,13,2,1,1,2,FALSE,yes
...,...,...,...,...,...,...,...,...,...
3856,September 2022,Male,60,2,1,1,2,TRUE,no
3857,September 2022,Male,62,2,1,1,2,FALSE,no
3858,September 2022,Male,63,2,2,1,1,TRUE,no
3859,September 2022,Male,68,2,2,2,1,TRUE,no


In [ ]:
# Export grouped_df as CSV file
grouped_df.to_csv("file/selected_value.csv", index=True)

In [17]:
# Convert "Date Flown" variables into date time
# merge_df["Date Flown"] = pd.to_datetime(merge_df["Date Flown"])

,ID,Gender,Age,Customer Type,Type of Travel,Class,Flight Distance,Departure Delay,Arrival Delay,Departure and Arrival Time Convenience,...,Baggage Handling,Satisfaction,Airline Name,Verified,Value For Money,Recommend,customer_type,travel_type,class_type,satis_level
Date Flown,,,,,,,,,,,,,,,,,,,,,
,472,472,472,472,472,472,472,472,472,472,...,472,472,472,472,472,472,472,472,472,472
April 2012,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
April 2015,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
April 2016,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
April 2017,33,33,33,33,33,33,33,33,33,33,...,33,33,33,33,33,33,33,33,33,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
September 2018,26,26,26,26,26,26,26,26,26,26,...,26,26,26,26,26,26,26,26,26,26
September 2019,74,74,74,74,74,74,74,74,74,74,...,74,74,74,74,74,74,74,74,74,74
September 2020,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
